# Map Reduce
- stuff
- refine
- Map reduce
- Map re-mark

# Map Reduce
- 先将每个文档或文档块分别投喂给LLM，并得到结果集（Map步骤），然后通过一个文档合并链，获得一个输出结果（Reduce步骤） 
![Map reduce](./img/map.png)

In [1]:
from langchain.chains import MapReduceDocumentsChain
from langchain.chains import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

#load pdf
loader = PyPDFLoader("loader.pdf")
docs = loader.load()
#split text
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=0,
)
split_docs = text_splitter.split_documents(docs)
#print(split_docs)

#map chain
map_template = """对以下文字做简洁的总结:
"{content}"
简洁的总结:"""
map_prompt = PromptTemplate.from_template(map_template)
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
map_chain = LLMChain(
    llm=llm,
    prompt=map_prompt,
)

#reduce chain
reduce_template = """以下是一个摘要集合:
{doc_summaries}
将上述摘要与所有关键细节进行总结.
总结:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(
    prompt=reduce_prompt,
    llm=llm,
)
stuff_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries",
)
reduce_final_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain,
    #超过4000个token就会切入到下一个stuff_chain
    collapse_documents_chain=stuff_chain,
    token_max=4000,
)

#map reduce chain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    document_variable_name="content",
    reduce_documents_chain=reduce_final_chain,
)

c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [2]:
summary = map_reduce_chain.run(split_docs)
print(summary)

c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


蒂法是《最终幻想VII》中的虚构角色，是克劳德的青梅竹马，拥有格斗技能，是雪崩组织成员。她与克劳德有特殊关系，帮助他找回记忆并一起击败敌人。蒂法展现出坚强、独立和有吸引力的形象，被认为是游戏中最好的女性角色之一。她在游戏中扮演重要角色，外观设计多变，备受好评，被评为电子游戏史上最伟大的女性角色之一，具有独立、性感和战斗能力。她的名字象征着爱情、美丽和自我牺牲。


# Map re-rank
- 先将每个文档或文档块投喂给LLM,并对每个文档或文档块生成问题的答案进行打分，然后将打分最高的文档或文档块作为最终答案返回


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
#load
loader = PyPDFLoader("loader.pdf")
docs = loader.load()
#split
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

chain = load_qa_with_sources_chain(
    ChatOpenAI(temperature=0), 
    chain_type="map_rerank", 
    metadata_keys=['source'], 
    return_intermediate_steps=True
)
print(chain)

llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=RegexParser(regex='(.*?)\\nScore: (\\d*)', output_keys=['answer', 'score']), template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHelpful Answer: red\nScore: 100\n\nExample #2\n\nContext:\n---------\

In [6]:
"""
Use the following pieces of context to answer the question in chinese at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n
In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\n
Question: [question here]\n
Helpful Answer: [answer here]\n
Score: [score between 0 and 100]\n\n
How to determine the score:\n
- Higher is a better answer\n
- Better responds fully to the asked question, with sufficient level of detail\n
- If you do not know the answer based on the context, that should be a score of 0\n
- Don't be overconfident!\n\n
Example #1\n\n
Context:\n
---------\n
Apples are red\n
---------\n
Question: what color are apples?\n
Helpful Answer: red\n
Score: 100\n\n
Example #2\n\n
Context:\n
---------\n
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv\n
---------\n
Question: what type was the car?\n
Helpful Answer: a sports car or an suv\n
Score: 60\n\n
Example #3\n\n
Context:\n---------\n
Pears are either red or orange\n
---------\n
Question: what color are apples?\n
Helpful Answer: This document does not answer the question\n
Score: 0\n\n
Begin!\n\n
Context:\n
---------\n
{context}\n
---------\n
Question: {question}\n
Helpful Answer:"""

"\nUse the following pieces of context to answer the question in chinese at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\n\nQuestion: [question here]\n\nHelpful Answer: [answer here]\n\nScore: [score between 0 and 100]\n\n\nHow to determine the score:\n\n- Higher is a better answer\n\n- Better responds fully to the asked question, with sufficient level of detail\n\n- If you do not know the answer based on the context, that should be a score of 0\n\n- Don't be overconfident!\n\n\nExample #1\n\n\nContext:\n\n---------\n\nApples are red\n\n---------\n\nQuestion: what color are apples?\n\nHelpful Answer: red\n\nScore: 100\n\n\nExample #2\n\n\nContext:\n\n---------\n\nit was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv\n\n---------\n\nQuestion: wh

In [7]:
query = "what is this document talk about?answer by chinese"
result = chain({"input_documents":split_docs,"question":query})
result

c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Programs\Python\Python312\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'input_documents': [Document(page_content='蒂法介绍蒂法·洛克哈特（⽇语：ティファ・ロックハート，Tifa Rokkuhāto，英语：Tifa Lockhart）为电⼦游戏《最终幻想VII》及《最终幻想VII补完计划》相关作品中的虚构⻆⾊，由野村哲也创作和设计，此后也在多个游戏中客串登场。2014年东京电玩展上，星名美津纪cosplay《最终幻想VII 降临之⼦》中的蒂法·洛克哈特蒂法是克劳德的⻘梅⽵⻢，两⼈同为尼布鲁海姆出身。在⽶德加经营作为反抗组织“雪崩”根据地的酒馆“第七天堂”，并且是⼩有名⽓的招牌店员。擅⻓格⽃，以拳套为武器。本传7年前克劳德离开故乡从军时，曾许下约定“如果有危机时⼀定会保护她”。与爱丽丝相识之后，两⼈成为好友。第⼀个察觉克劳德记忆混乱的⼈，后来协助精神崩溃的克劳德重新找回真正的⾃⼰。本传的⼤战结束后，依⼤家的期待在战后新⽣的⽶德加再次开设第七天堂（原第七天堂因第柒区圆盘崩塌遭压毁），同时也照顾⼀群受到星痕症候群折磨的孩⼦们。蒂法被《纽约时报》称为“⽹络⼀代”的海报⼥郎，与劳拉·克罗夫特相⽐，她是电⼦游戏中坚强、独⽴和有吸引⼒的⼥性⻆⾊的典型代表。媒体普遍称赞其实⼒和外表，并称她为游戏世界中最好的⼥性⻆⾊之⼀。在《最终幻想VII》本传中，蒂法年龄20岁、身⾼167cm、⽣⽇5⽉3⽇、⾎型B型、出⽣地尼布尔海姆。登场《最终幻想VII》蒂法在《最终幻想VII》原版中⾸次亮相，是克劳德的⻘梅⽵⻢、第七天堂酒吧的看板娘、极端环境组织“雪崩”成员，该组织反抗巨型企业“神罗”因其⼤量抽取魔晄⽤作动⼒能源。在注意到克劳德的性格改变后，她说服克劳德加⼊雪崩，以密切关注他，并且跟随他追寻游戏中的对⼿萨菲罗斯。她⽆法阻⽌克劳德被萨菲罗斯操纵，在他的精神崩溃后，她帮助克劳德康复，并且两⼈意识到彼此间的相互感觉，最后与伙伴们⼀同击败了萨菲罗斯。[2]在闪回中可知，⼉时的蒂法⼀直是村中⼩孩的⼈⽓王。在⺟亲过世后，思念⺟亲的蒂法决定沿着⼩路⾛到他们故乡尼布尔海姆附近的⼀座⼭上，认为这样就能⻅到过世的⺟亲，原本跟着蒂法的其他⼩孩都在半路上因害怕⽽放弃，唯独克劳德仍坚定的在后⾯跟随，希望能在危机时保护蒂法。然⽽，他们俩都从⼭上跌落受伤，蒂法昏迷了⼀个星期，她的⽗亲认为克劳德对此负有责任[3]，甚为严令禁⽌克劳德再接近蒂法，但蒂法反